# Lending Club Case Study

## Import packages

In [668]:
#Import necessary packages
import chardet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Load dataset

In [669]:
#check the encoding of input data:
with open('loan.csv','rb') as raw_data:
    result = chardet.detect(raw_data.read(2000))
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [670]:
#load the loan dataset
loan_data = pd.read_csv('loan.csv',dtype='unicode')

In [671]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [672]:
loan_data.shape

(39717, 111)

## Data cleaning

### Removing unnecessary and null columns

In [673]:
loan_data.isnull().sum()

id                                    0
member_id                             0
loan_amnt                             0
funded_amnt                           0
funded_amnt_inv                       0
term                                  0
int_rate                              0
installment                           0
grade                                 0
sub_grade                             0
emp_title                          2459
emp_length                         1075
home_ownership                        0
annual_inc                            0
verification_status                   0
issue_d                               0
loan_status                           0
pymnt_plan                            0
url                                   0
desc                              12940
purpose                               0
title                                11
zip_code                              0
addr_state                            0
dti                                   0


There are columns with many Null values. We can drop these rows as they won't be of use to us in the analysis. We will drop rows that contain more that 80 percent as null values.

In [674]:
null_percent = loan_data.isnull().sum() * 100 / len(loan_data)
null_percent

id                                  0.000000
member_id                           0.000000
loan_amnt                           0.000000
funded_amnt                         0.000000
funded_amnt_inv                     0.000000
term                                0.000000
int_rate                            0.000000
installment                         0.000000
grade                               0.000000
sub_grade                           0.000000
emp_title                           6.191303
emp_length                          2.706650
home_ownership                      0.000000
annual_inc                          0.000000
verification_status                 0.000000
issue_d                             0.000000
loan_status                         0.000000
pymnt_plan                          0.000000
url                                 0.000000
desc                               32.580507
purpose                             0.000000
title                               0.027696
zip_code  

In [675]:
#Get the columns that does has more than 80% null values. 
null_percent_df = pd.DataFrame({'column_name': loan_data.columns,
                               'null_percent': null_percent.round(2)})
(null_percent_df.null_percent > 80).sum()

56

There seems to be 56 columns with percent of null values exceeding 80 percent. We can drop these columns.

In [676]:
#get the column names to be dropped.
columns_to_drop = null_percent_df[null_percent_df.null_percent > 80].column_name.tolist()
master_frame = loan_data.drop(columns_to_drop, axis = 1)
master_frame.shape

(39717, 55)

In [677]:
#checking the percent of null after removal of null columns
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

emp_title                      6.191303
emp_length                     2.706650
desc                          32.580507
title                          0.027696
mths_since_last_delinq        64.662487
revol_util                     0.125891
last_pymnt_d                   0.178765
last_credit_pull_d             0.005036
collections_12_mths_ex_med     0.140998
chargeoff_within_12_mths       0.140998
pub_rec_bankruptcies           1.754916
tax_liens                      0.098195
dtype: float64

In [678]:
master_frame['chargeoff_within_12_mths'].value_counts()

0    39661
Name: chargeoff_within_12_mths, dtype: int64

In [679]:
master_frame['tax_liens'].value_counts()

0    39678
Name: tax_liens, dtype: int64

In [680]:
master_frame['collections_12_mths_ex_med'].value_counts()

0    39661
Name: collections_12_mths_ex_med, dtype: int64

In [681]:
master_frame['policy_code'].value_counts()

1    39717
Name: policy_code, dtype: int64

In [682]:
master_frame['pymnt_plan'].value_counts()

n    39717
Name: pymnt_plan, dtype: int64

In [683]:
master_frame['delinq_amnt'].value_counts()

0    39717
Name: delinq_amnt, dtype: int64

In [684]:
master_frame['acc_now_delinq'].value_counts()

0    39717
Name: acc_now_delinq, dtype: int64

In [685]:
master_frame['application_type'].value_counts()

INDIVIDUAL    39717
Name: application_type, dtype: int64

In [686]:
master_frame['initial_list_status'].value_counts()

f    39717
Name: initial_list_status, dtype: int64

### Unnecessary columns:

desc: Loan description provided by the borrower. This is a text field contains descirptive data. Processing this is beyond the scope of this assingment. The issued month can be got from the 'issue_d' field.

url: the URL does not help with the analysis 

chargeoff_within_12_mths: 'chargeoff_within_12_mths' only has '0' and null and does not contribute to the analysis.

tax_liens: 'tax_liens' only has '0' and null and does not contribute to the analysis.

collections_12_mths_ex_med: 'collections_12_mths_ex_meds' only has '0' and null and does not contribute to the analysis.

policy_code: Has only value '1'

pymnt_plan: has only value 'n'

delinq_amnt: has only value '0'

acc_now_delinq: has only value '0'

application_type: has only type 'INDIVIDUAL'

initial_list_status: has only value 'f'

title: Has inconsistet text data. It requires to NLP to make it meaningful categories.

emp_title: Has inconsistet text data. It requires to NLP to make it meaningful categories.

id: doesn't affect the analysis

member_id: doesn't affect the analysis


In [687]:
#dropping the columsn mentioned above
drop_list = ['desc','tax_liens','chargeoff_within_12_mths','collections_12_mths_ex_med','url',
            'policy_code','pymnt_plan','delinq_amnt','acc_now_delinq','application_type',
            'initial_list_status','title','emp_title','id','member_id']
master_frame = master_frame.drop(drop_list, axis = 1)

In [688]:
master_frame.shape

(39717, 40)

In [689]:
#checking the percent of null after removal of the above columns
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

emp_length                 2.706650
mths_since_last_delinq    64.662487
revol_util                 0.125891
last_pymnt_d               0.178765
last_credit_pull_d         0.005036
pub_rec_bankruptcies       1.754916
dtype: float64

In [690]:
#number of rows having atleast one NaN value
print (master_frame.isna().any(axis=1).sum())

26752


In [691]:
#Percent of NaN values by row
null_count_row = master_frame.isnull().sum(axis = 1)*100/len(master_frame.columns)
#null_count_row.sort(reverse = True)
#null_count_row.value_counts()
null_count_row[null_count_row > 0].sort_values(ascending=False)

4714     7.5
28547    7.5
36209    7.5
13891    7.5
14839    7.5
        ... 
26502    2.5
26504    2.5
26505    2.5
26507    2.5
0        2.5
Length: 26752, dtype: float64

The percent of null values in each row is less than 10 percent. So we will leave it as is and try to impute the remaining null values for the columns.

### Unnecessary rows

In [692]:
master_frame.loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

We don't need rows with loan_status = 'Current' as the borrowers are still paying the loan.

In [693]:
master_frame = master_frame[master_frame.loan_status != 'Current']

In [694]:
#checking the percent of null after removal of the above columns
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

emp_length                 2.677761
mths_since_last_delinq    64.559193
revol_util                 0.129611
last_pymnt_d               0.184047
last_credit_pull_d         0.005184
pub_rec_bankruptcies       1.806776
dtype: float64

mths_since_last_delinq still has a considerable amount of null values. Imputing the columns will not give any meaning full information. So we can drop it.

In [695]:
master_frame = master_frame.drop(['mths_since_last_delinq'], axis = 1)

In [696]:
#Finding number of unique values in each column. We can remove columns that have 1 value as it
#doesn't affect our analysis
col_count = []
for col in master_frame.columns.tolist():
    col_count.append({'column': col, 'unique_count': len(master_frame[col].unique())})
unique = pd.DataFrame(col_count).sort_values('unique_count')
unique.head(19)

,column,unique_count
27,out_prncp_inv,1
26,out_prncp,1
3,term,2
13,loan_status,2
11,verification_status,3
38,pub_rec_bankruptcies,4
22,pub_rec,5
9,home_ownership,5
6,grade,7
20,inq_last_6mths,9


In [697]:
master_frame.out_prncp.value_counts()

0    38577
Name: out_prncp, dtype: int64

In [698]:
master_frame.out_prncp_inv.value_counts()

0    38577
Name: out_prncp_inv, dtype: int64

In [699]:
#delete the above two columns as they contain only 1 value ('0') after deleting "Current" loan_status
master_frame = master_frame.drop(['out_prncp','out_prncp_inv'], axis = 1)

### Imputing/cleaning the columns for format,type  and missing values 

In [700]:
#checking the percent of null after removal of the above columns and rows
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

emp_length              2.677761
revol_util              0.129611
last_pymnt_d            0.184047
last_credit_pull_d      0.005184
pub_rec_bankruptcies    1.806776
dtype: float64

In [701]:
master_frame.emp_length.value_counts()

10+ years    8488
< 1 year     4508
2 years      4291
3 years      4012
4 years      3342
5 years      3194
1 year       3169
6 years      2168
7 years      1711
8 years      1435
9 years      1226
Name: emp_length, dtype: int64

In [702]:
master_frame.emp_length.isna().sum()

1033

In [703]:
# remove the 'years' from the employement length and make it numeric
#del master_frame['emp_len_yrs']
master_frame['emp_len_yrs'] = master_frame.emp_length.str.rstrip('years ')
master_frame['emp_len_yrs'][master_frame['emp_len_yrs'] == '10+'] = '10' 
master_frame['emp_len_yrs'][master_frame['emp_len_yrs'] == '< 1'] = '1' 
#We will make Na entires as it's own category => 0
master_frame['emp_len_yrs'][master_frame['emp_len_yrs'].isna()] = '0' 
master_frame.emp_len_yrs = master_frame.emp_len_yrs.apply(int)
del master_frame['emp_length']

In [704]:
master_frame.emp_len_yrs.value_counts()

10    8488
1     7677
2     4291
3     4012
4     3342
5     3194
6     2168
7     1711
8     1435
9     1226
0     1033
Name: emp_len_yrs, dtype: int64

In [705]:
#Make the following columns numeric
int_col = ['total_acc']
float_col = ['loan_amnt','funded_amnt','funded_amnt_inv','installment',
             'annual_inc','revol_bal','total_pymnt','total_pymnt_inv',
             'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries',
             'collection_recovery_fee','last_pymnt_amnt']
master_frame[float_col] = master_frame[float_col].apply(lambda x: x.astype(float))
master_frame[int_col] = master_frame[int_col].apply(lambda x: x.astype('int64'))

In [706]:
#split issue_d field to month and year.
master_frame['issue_d_mnth'] = master_frame.issue_d.apply(lambda x: x.split('-')[0])
print('number of unique months: ',len(master_frame.issue_d_mnth.value_counts()))
master_frame['issue_d_year'] = master_frame.issue_d.apply(lambda x: '20' + x.split('-')[1])
print('number of unique years: ',len(master_frame.issue_d_year.value_counts()))
del master_frame['issue_d']

number of unique months:  12
number of unique years:  5


In [707]:
master_frame.issue_d_year.value_counts()

2011    20516
2010    11532
2009     4716
2008     1562
2007      251
Name: issue_d_year, dtype: int64

In [708]:
print('max in DTI: ',master_frame.dti.max())
print('min in DTI: ',master_frame.dti.min())

max in DTI:  9.99
min in DTI:  0


In [709]:
#DTI: convert to numeric and perform binning
'''
0   - 5      => 5   
>5  - 10     => 10
>10 - 15     => 15
>15 - 20     => 20
>20 - 25     => 25
>25 - 30     => 30

We'll stop at 30 because 29.99 is the max value.

NOTE: lower DTI is better.
'''
master_frame.dti = master_frame.dti.apply(float)
master_frame['dti_range'] =master_frame['dti']
#Ignoring the SettingWithCopyWarning.
with pd.option_context('mode.chained_assignment', None):
    master_frame['dti_range'][(master_frame['dti_range'] <= 5)] = 5
    master_frame['dti_range'][(master_frame['dti_range'] > 5) & (master_frame['dti_range'] <= 10)] = 10
    master_frame['dti_range'][(master_frame['dti_range'] > 10) & (master_frame['dti_range'] <= 15)] = 15
    master_frame['dti_range'][(master_frame['dti_range'] > 15) & (master_frame['dti_range'] <= 20)] = 20
    master_frame['dti_range'][(master_frame['dti_range'] > 20) & (master_frame['dti_range'] <= 25)] = 25
    master_frame['dti_range'][(master_frame['dti_range'] > 25) & (master_frame['dti_range'] <= 30)] = 30
del master_frame['dti']
master_frame.dti_range = master_frame.dti_range.apply(int)

In [710]:
#split earliest_cr_line field to month and year.
master_frame['earliest_cr_line_mnth'] = master_frame.earliest_cr_line.apply(lambda x: x.split('-')[0])
print('number of unique months: ',len(master_frame.earliest_cr_line_mnth.value_counts()))
master_frame['earliest_cr_line_year'] = master_frame.earliest_cr_line.apply(lambda x: x.split('-')[1])
print('number of unique years: ',len(master_frame.earliest_cr_line_year.value_counts()))
del master_frame['earliest_cr_line']

number of unique months:  12
number of unique years:  53


In [711]:
master_frame['revol_util_percent'] = master_frame['revol_util'].str.rstrip('%')
master_frame.revol_util_percent = master_frame.revol_util_percent.apply(float)
del master_frame['revol_util']

In [712]:
master_frame.total_pymnt.value_counts()

11196.569430    26
0.000000        16
10956.775960    16
11784.232230    16
13148.137860    15
                ..
11552.630760     1
5862.194934      1
28376.870000     1
8457.317962      1
12179.013410     1
Name: total_pymnt, Length: 36714, dtype: int64

In [713]:
master_frame.dtypes

loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
home_ownership              object
annual_inc                 float64
verification_status         object
loan_status                 object
purpose                     object
zip_code                    object
addr_state                  object
delinq_2yrs                 object
inq_last_6mths              object
open_acc                    object
pub_rec                     object
revol_bal                  float64
total_acc                    int64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int              float64
total_rec_late_fee         float64
recoveries                 float64
collection_recovery_fee    float64
last_pymnt_d        

In [714]:
#checking the percent of null after removal of the above columns and rows
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

last_pymnt_d            0.184047
last_credit_pull_d      0.005184
pub_rec_bankruptcies    1.806776
revol_util_percent      0.129611
dtype: float64

In [715]:
#checking the most frequently occuring value
master_frame.pub_rec_bankruptcies.describe()

count     37880
unique        3
top           0
freq      36238
Name: pub_rec_bankruptcies, dtype: object

In [716]:
#most of the pub_rec are 0 so we can fill the missing values with this.
master_frame['pub_rec_bankruptcies'] = master_frame['pub_rec_bankruptcies'].fillna(0)

In [717]:
master_frame.revol_util_percent.describe()

count    38527.000000
mean        48.702777
std         28.364741
min          0.000000
25%         25.200000
50%         49.100000
75%         72.300000
max         99.900000
Name: revol_util_percent, dtype: float64

In [718]:
#checking the percent of null after removal of the above columns and rows
new_null_percent = master_frame.isnull().sum() * 100 / len(master_frame)
new_null_percent[new_null_percent>0]

last_pymnt_d          0.184047
last_credit_pull_d    0.005184
revol_util_percent    0.129611
dtype: float64